In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr
import talib

In [3]:
MONTH = 21
YEAR = 12 * MONTH

In [4]:
START = '2013-01-01'
END = '2017-12-31'

sns.set_style('whitegrid')
idx = pd.IndexSlice

In [5]:
df = (pd.read_csv(r'C:\Users\Wendel\OneDrive\Machine-Learning-for-Algorithmic-Trading-Second-Edition\data\wiki_prices\teste.csv',
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True)
                 .sort_index())[{'adj_open','adj_high', 'adj_low', 'adj_close', 'adj_volume'}]

In [17]:
prices = df.copy()
prices

adj_low   adj_open  adj_close   adj_high    adj_volume
date       ticker                                                          
1962-01-02 ARNC     3.448668   3.458163   3.448668   3.468185  4.480000e+04
           BA       0.871816   0.887160   0.871816   0.887160  3.521981e+05
           CAT      1.562791   1.578370   1.578370   1.593539  1.632000e+05
           DD       4.718414   4.718414   4.718414   4.772144  3.600000e+04
           DIS      0.141259   0.141259   0.141259   0.145999  4.088582e+05
...                      ...        ...        ...        ...           ...
2018-03-27 ZIXI     4.350000   4.430000   4.390000   4.450000  2.073040e+05
           ZNGA     3.680000   3.850000   3.700000   3.850000  1.102601e+07
           ZOES    14.750000  15.130000  14.880000  15.240000  2.612420e+05
           ZTS     81.210000  84.080000  81.860000  84.450000  2.676191e+06
           ZUMZ    23.350000  24.650000  23.600000  24.650000  4.038840e+05

[15389314 rows x 5 columns]

In [23]:
prices = prices.loc[ ('2000-01'):('2018-03-27'), :]
prices.dropna(inplace=True)
prices.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11343366 entries, (Timestamp('2000-01-03 00:00:00'), 'A') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 5 columns):
 #   Column      Non-Null Count     Dtype  
---  ------      --------------     -----  
 0   adj_low     11343366 non-null  float64
 1   adj_open    11343366 non-null  float64
 2   adj_close   11343366 non-null  float64
 3   adj_high    11343366 non-null  float64
 4   adj_volume  11343366 non-null  float64
dtypes: float64(5)
memory usage: 476.7+ MB


In [33]:
#Computer rolling average dollar volume
prices['dollar_vol'] = prices.loc[:, 'adj_close'].mul(prices.loc[:, 'adj_volume'], axis=0)
prices

adj_low   adj_open  adj_close   adj_high    adj_volume  \
date       ticker                                                             
2000-01-03 A       45.969377  53.726454  49.121329  53.856080  3.343600e+06   
           AAN      4.975857   5.117056   4.975857   5.117056  2.080012e+04   
           AAON     1.025920   1.120912   1.025920   1.120912  6.836995e+05   
           AAPL     3.267146   3.369314   3.596463   3.614454  1.339492e+08   
           ABAX     7.007472   7.299450   7.063984   7.421892  5.310000e+04   
...                      ...        ...        ...        ...           ...   
2018-03-27 ZIXI     4.350000   4.430000   4.390000   4.450000  2.073040e+05   
           ZNGA     3.680000   3.850000   3.700000   3.850000  1.102601e+07   
           ZOES    14.750000  15.130000  14.880000  15.240000  2.612420e+05   
           ZTS     81.210000  84.080000  81.860000  84.450000  2.676191e+06   
           ZUMZ    23.350000  24.650000  23.600000  24.650000  4.038840e+05   

                     dollar_vol  
date       ticker                
2000-01-03 A       1.642421e+08  
           AAN     1.034984e+05  
           AAON    7.014208e+05  
           AAPL    4.817433e+08  
           ABAX    3.750975e+05  
...                         ...  
2018-03-27 ZIXI    9.100646e+05  
           ZNGA    4.079624e+07  
           ZOES    3.887281e+06  
           ZTS     2.190730e+08  
           ZUMZ    9.531662e+06  

[11343366 rows x 6 columns]

In [35]:
prices['dollar_vol'] = (prices.groupby('ticker').dollar_vol.rolling(window=21).mean().fillna(0).reset_index(level=0, drop=True))

In [36]:
prices

adj_low   adj_open  adj_close   adj_high    adj_volume  \
date       ticker                                                             
2000-01-03 A       45.969377  53.726454  49.121329  53.856080  3.343600e+06   
           AAN      4.975857   5.117056   4.975857   5.117056  2.080012e+04   
           AAON     1.025920   1.120912   1.025920   1.120912  6.836995e+05   
           AAPL     3.267146   3.369314   3.596463   3.614454  1.339492e+08   
           ABAX     7.007472   7.299450   7.063984   7.421892  5.310000e+04   
...                      ...        ...        ...        ...           ...   
2018-03-27 ZIXI     4.350000   4.430000   4.390000   4.450000  2.073040e+05   
           ZNGA     3.680000   3.850000   3.700000   3.850000  1.102601e+07   
           ZOES    14.750000  15.130000  14.880000  15.240000  2.612420e+05   
           ZTS     81.210000  84.080000  81.860000  84.450000  2.676191e+06   
           ZUMZ    23.350000  24.650000  23.600000  24.650000  4.038840e+05   

                     dollar_vol  
date       ticker                
2000-01-03 A       0.000000e+00  
           AAN     0.000000e+00  
           AAON    0.000000e+00  
           AAPL    0.000000e+00  
           ABAX    0.000000e+00  
...                         ...  
2018-03-27 ZIXI    1.196128e+06  
           ZNGA    3.752806e+07  
           ZOES    4.870005e+06  
           ZTS     2.028849e+08  
           ZUMZ    9.044512e+06  

[11343366 rows x 6 columns]